# Natural Language Processing - Apple Sentiment
by Michael Kearns

# Business Understanding

Customer relations and approval is highly valued at the Tech Sales Company (TSC). Currently, TSC sells Apple products and we want to be sure if we should continue to market and supply Apple products to our customers. If customers no longer like Apple, we want to separate ourselves from the brand and show our loyalty to our customers and promote other products. To track what customers do not like about Apple, we plan to develop a natural language processing model that can identify if a customer's post is critical and should be reviewed later. Non critical posts will be considered posts that do not provide negative or useful information that can help show what our customers do not like about Apple. 

# Data Understanding
To train and test this machine learning model, data from [Crowdflower](https://www.kaggle.com/datasets/slythe/apple-twitter-sentiment-crowdflower) will be used. This dataset includes nearly 4,000 X posts that reference Apple from December, 2014. The primary features that will be used are the "Sentiment" and "Text" features that include the posts and that user's sentiment toward Apple, rated from 1-3 scale


## Data Preparation

In [1]:
import pandas as pd
import numpy as np

#import csv file
filename = 'data/Apple-Twitter-Sentiment-DFE.csv'
df = pd.read_csv(filename, encoding="latin1")

#check dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _unit_id              3886 non-null   int64  
 1   _golden               3886 non-null   bool   
 2   _unit_state           3886 non-null   object 
 3   _trusted_judgments    3886 non-null   int64  
 4   _last_judgment_at     3783 non-null   object 
 5   sentiment             3886 non-null   object 
 6   sentiment:confidence  3886 non-null   float64
 7   date                  3886 non-null   object 
 8   id                    3886 non-null   float64
 9   query                 3886 non-null   object 
 10  sentiment_gold        103 non-null    object 
 11  text                  3886 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 337.9+ KB


Only the "sentiment" and "text" columns will be retained for this model. All other columns can be removed.

In [2]:
#Keep relevant columns in dataset.
df = df[['sentiment','text']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  3886 non-null   object
 1   text       3886 non-null   object
dtypes: object(2)
memory usage: 60.8+ KB


In [3]:
#Check overall sentiment distribution
print(df['sentiment'].value_counts())

sentiment
3               2162
1               1219
5                423
not_relevant      82
Name: count, dtype: int64


Based on background information provided by the datasource, sentiment is ranked as 1 - Negative, 3 - Neutral, 5 - Positive.

In [4]:
#Check proportion of sentiment values
print(df['sentiment'].value_counts(normalize = True))

sentiment
3               0.556356
1               0.313690
5               0.108852
not_relevant    0.021101
Name: proportion, dtype: float64


"Neutral" sentiment posts take up almost 56% of the datset, followed by %31 "Negative" posts, and %11 "Positive" posts. If the model was created based on this current distribution, the model would likely be more influenced by the "Neutral" or "Negative" posts and will not be able to accurately rate future posts. Therefore, some tactics will need to be implemented to deal with the class imbalance. Before the class imbalance is addressed, the data will need to be cleaned/preprocessed and split into train and test sets.

There are less than 100 posts labeled as "not_relevant". These can be removed and will not be considered in the model.

In [5]:
#Remove "not_relevant" rows
df = df[df['sentiment']!= 'not_relevant']

#Recheck proportion of sentiment values and confirm "not_relevant" rows are removed
print(df['sentiment'].value_counts(normalize = True))

sentiment
3    0.568349
1    0.320452
5    0.111199
Name: proportion, dtype: float64


In [6]:
#convert sentiment to integers
df['sentiment'] = df['sentiment'].astype(int)

In [7]:
#Randomly convert 800 of the 3's to 5's

# Step 1: Identify rows with a rating of 3
rating_3_indices = df[df['sentiment'] == 3].index

# Step 2: select random group of 600 ratings = 3
np.random.seed(42)
random_subset_indices = np.random.choice(rating_3_indices, size=800, replace=False)

# Step 3: Convert random subset to value = 5
df['sentiment'].loc[random_subset_indices] = 5

#Step 4: Remove remaining rows from df
remaining_indices = df[df['sentiment'] == 3].index
df = df.drop(remaining_indices)

/var/folders/sk/8hr8rsms4cb5y3ryy9w5yplh0000gn/T/ipykernel_67462/2412120355.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'].loc[random_subset_indices] = 5


In [8]:
#Check overall sentiment distribution
print(df['sentiment'].value_counts())

sentiment
5    1223
1    1219
Name: count, dtype: int64


Data needs to simplified and cleaned. Initial steps to preprocess the text will be to convert all letters in a post to lowercase. Mentiones, ie. @..., and URLS will be removed. The user mentions will be primarily directed at Apple which is not signifcant as all posts in this data are in reference towards Apple. URLS are long and not semantic content.

In [ ]:
#Convert all letters to lowercase
df['text_cleaned'] = df['text'].str.lower()

import re

def clean_text(text):
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = text.strip()  # Remove extra whitespaces
    return text

# Apply function to the dataset
df['text_cleaned'] = df['text_cleaned'].apply(clean_text)

# Exploratory Data Analysis

To make the text more suitable for a machine learning model, the text needs to be tokenized. This will be done using the **nltk** module.

In [ ]:
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import string

In [ ]:
#tokenize text using basic regex pattern
token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(token_pattern)

df['text_tokenized']=df['text_cleaned'].apply(tokenizer.tokenize)

In [ ]:
#Create Frequency Distribution of Tokens
text_freq_dist = FreqDist(df['text_tokenized'].explode())

# Plot the top 10 tokens

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def visualize_top_10(freq_dist, title):

    # Extract data for plotting
    top_10 = list(zip(*freq_dist.most_common(10)))
    tokens = top_10[0]
    counts = top_10[1]

    # Set up plot and plot data
    fig, ax = plt.subplots(figsize = (5,2))
    ax.bar(tokens, counts)

    # Customize plot appearance
    ax.set_title(title)
    ax.set_ylabel("Count")
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.tick_params(axis="x", rotation=90)
    
visualize_top_10(text_freq_dist, 'Top 10 Word Frequency')

In [ ]:
# Add in labels for filtering (we won't pass them in to the model)
def visualize_top_ten_tokens_per_sentiment(sentiment,rating,text):
    freq_dist = FreqDist(df[df[sentiment] == rating][text].explode())
    if rating == 1:
        title = 'Top 10 Word Frequency - Negative Posts'
    elif rating == 3:
        title = 'Top 10 Word Frequency - Neutral Posts'
    else: title = 'Top 10 Word Frequency - Positive Posts'

    return  visualize_top_10(freq_dist, title)

for i in range(1, 7, 2):
    visualize_top_ten_tokens_per_sentiment('sentiment',i,'text_tokenized')
    

Stopwords seem to be very repeatable among the three types of posts. They should me removed to make the tokens more distinct.

In [ ]:
#download stopwords to remove
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')
#'aapl' and 'apple' will be very repetive in these posts so they will be removed.
stopwords_list.append('apple')
stopwords_list.append('aapl')
stopwords_list.append('rt')

def remove_stopwords(token_list):
    stopwords_delete = [token for token in token_list if token not in stopwords_list]
    return stopwords_delete

df['text_stopwords_removed'] = df['text_tokenized'].apply(remove_stopwords)

In [ ]:
for i in range(1, 7, 2):
    visualize_top_ten_tokens_per_sentiment('sentiment',i,'text_stopwords_removed')

# Modeling

## Baseline Model 

For the baseline model a Naive Bayes Model will be used. For this case, 

In [ ]:
#split the features and target into train and test sets for modeling.

from sklearn.model_selection import train_test_split

y = df['sentiment']
X = df.drop('sentiment',axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 30)

In [ ]:
print(y_train.value_counts())

As stated in the Business Understanding, the model should be able to deterine if the post is critical or not to be reviewed. Therefore, this target needs to be converted to a binary distribution. Target with ratings of 1 will be considered "Critical" and ratings with 5 will be considered "Not Critical".

In [ ]:
#Randomly convert 600 of the 3's to and 5's

# Step 1: Identify rows with a rating of 3
rating_3_indices = y_train[y_train == 3].index

# Step 2: select random group of 600 ratings = 3
np.random.seed(42)
random_subset_indices = np.random.choice(rating_3_indices, size=600, replace=False)

# Step 3: Convert random subset to value = 5
y_train_binary = y_train
y_train_binary.loc[random_subset_indices] = 5

#Step 4: Remove remaining rows from y_train
remaining_indices = y_train[y_train == 3].index
y_train_binary = y_train_binary.drop(remaining_indices)

#Step 5: Remove rows from X_train_vectorized
X_train_binary = X_train.drop(remaining_indices)

In [ ]:
# Vectorize the features
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate a vectorizer with max_features=10
# (we are using the default token pattern)
tfidf = TfidfVectorizer(
    max_features=10,
    stop_words=stopwords_list,
    token_pattern = token_pattern)

# Fit the vectorizer on X_train["text_cleaned"] and transform it
X_train_vectorized = tfidf.fit_transform(X_train_binary['text_cleaned'])

# Visually inspect the 10 most common words
pd.DataFrame.sparse.from_spmatrix(X_train_vectorized, columns=tfidf.get_feature_names_out())

In [ ]:
print(y_train_binary.value_counts())

In [ ]:
# use SMOTE to resample data.
#from imblearn.over_sampling import SMOTE
#smote = SMOTE(random_state = 35)
#X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized,y_train_binary)

In [ ]:
#Confirm resampled distribution
#print(y_train_resampled.value_counts())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Instantiate a MultinomialNB classifier
baseline_model = MultinomialNB()

# Evaluate the classifier on X_train_resampled, y_train_resampled
baseline_cv = cross_val_score(baseline_model, X_train_vectorized, y_train_binary)
baseline_cv

The baseline model is getting approximately 47-57% accuracy. Now that this is a binary model, it is performing about as well as if we were to just guess. This model should be improved if it wants to be useful.

## Model 2: Stemming Words

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language="english")

def stem_and_tokenize(document):
    tokens = tokenizer.tokenize(document)
    return [stemmer.stem(token) for token in tokens]

stemmed_stopwords = [stemmer.stem(word) for word in stopwords_list]

tfidf = TfidfVectorizer(max_features = 10,
                        stop_words = stemmed_stopwords,
                        tokenizer = stem_and_tokenize,
                        )

X_train_vectorized = tfidf.fit_transform(X_train_binary['text_cleaned'])
#X_train_resampled, y_train_resampled = smote.fit_resample(X_train_resampled, y_train_resampled)

# Evaluate the model classifier on X_train_resampled, y_train_resampled with lemmatized tokens
stemmed_cl_cv = cross_val_score(baseline_model, X_train_vectorized, y_train_binary)
stemmed_cl_cv


There is slight improvement as we are getting closer to 53% accuracy.

## Model 4: Random Forest Model

In [ ]:
#Make Baseline Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# Initialize the classifier
rf_model = RandomForestClassifier(n_estimators=10, random_state=42)

rf_model_cv = cross_val_score(rf_model, X_train_vectorized, y_train_binary)
rf_model_cv

61% Accuracy

In [ ]:
#Use GridSearchCv to determine optimal parameters for RF model
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator= rf_model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           verbose=2,
                           n_jobs=-1)

# Fit the model
grid_search.fit(X_train_vectorized, y_train_binary)

# Output best parameters
print("Best Parameters:", grid_search.best_params_)


In [ ]:
#Test RF model with optimized parameter
rf_model_optimized = RandomForestClassifier(random_state = 24,
                                           max_depth = 10,
                                           min_samples_leaf = 2,
                                           min_samples_split = 2,
                                           n_estimators = 50)
rf_model_optimized_cv = cross_val_score(rf_model_optimized, X_train_vectorized, y_train_binary)
rf_model_optimized_cv

Up to 61% accuracy

In [ ]:
def count_words(token_list):
    count_words = len(token_list)
    return count_words

num_words = X_train_binary['text_stopwords_removed'].apply(count_words)
num_words = num_words.reset_index(drop=True)

In [ ]:
tfidf = TfidfVectorizer(max_features = 10,
                        stop_words = stemmed_stopwords,
                        tokenizer = stem_and_tokenize,
                        )

X_train_vectorized = tfidf.fit_transform(X_train_binary['text_cleaned'])

# Create a full df of vectorized + engineered features
X_train_vectorized_df = pd.DataFrame(X_train_vectorized.toarray(), columns=tfidf.get_feature_names_out())
preprocessed_X_train = pd.concat([
    X_train_vectorized_df, num_words
], axis=1)
preprocessed_X_train

In [ ]:
rf_model_optimized_cv = cross_val_score(rf_model_optimized, preprocessed_X_train, y_train_binary)
rf_model_optimized_cv

## Final Model

In [ ]:
#Instantiate final model
final_model = rf_model_optimized
final_model.fit(X_train_vectorized, y_train_binary)
final_model.score(X_train_vectorized, y_train_binary)

In [ ]:
#Check accuracy on test results
final_X_test = tfidf.fit_transform(X_test['text_cleaned'])

#Convert y_test 3's to 5's

# Step 1: Identify rows with a rating of 3
rating_3_indices = y_test[y_test == 3].index

# Step 2: Assign these or 5
# Generate random integers (1 or 5) for each index in `rating_3_indices`
#np.random.seed(42)
new_ratings = np.full(len(rating_3_indices), 5)

# Step 3: Replace the values in the dataset
y_test_binary = y_test
y_test_binary.loc[rating_3_indices] = new_ratings

final_model.score(final_X_test,y_test)

In [ ]:
#Make Confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

y_pred = final_model.predict(final_X_test)

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',xticklabels = ['Critical','Not Critical'],yticklabels = ['Critical','Not Critical'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show();

# Conclusion

## Limitations

## Recommendations

## Next Steps